In [ ]:
import pandas as pd
import numpy as np

In [ ]:
age_at_scan = pd.ExcelFile("/home/a.kidane@mydre.org/brickstudy/secret_data/BRICK_datums_scans_clean_17072024 _versie_2.xlsx", engine = "openpyxl")


In [ ]:
sheet_names = age_at_scan.sheet_names

In [ ]:
print("Sheet names:", sheet_names)

In [ ]:
age_at_scan_2 = pd.read_excel("/home/a.kidane@mydre.org/brickstudy/secret_data/BRICK_datums_scans_clean_17072024 _versie_2.xlsx", sheet_name = "Dag van MRI+NPO overzicht ", engine = "openpyxl")
brick_gender = pd.read_excel("/home/a.kidane@mydre.org/brickstudy/secret_data/Gender_BRICKers23072024.xlsx", sheet_name = "Sheet1", engine = "openpyxl")
print(age_at_scan_2.head())

In [ ]:
#just to be sure, trim whitespace in the column names and the column "BRICK-nummer"
age_at_scan_2.columns = age_at_scan_2.columns.str.strip()

age_at_scan_2["BRICK-nummer"] = age_at_scan_2["BRICK-nummer"].str.strip()
print(age_at_scan_2.head())

In [ ]:
#remove the "00:00:00" by removing characters in "Datum_NPO_T0"
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].astype(str)
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].str.strip()
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].str[:-9]

print(age_at_scan_2.head())

In [ ]:
#rename second column to gender
brick_gender.rename(columns={brick_gender.columns[1]: "Gender"}, inplace=True)
print(brick_gender.head())

In [ ]:
#now make the hyphen a underscore up age at scan csv with this table
brick_gender["Participant ID"] = brick_gender["Participant ID"].str.replace('-',"_")
print(brick_gender.head())

In [ ]:
#prepare merging, first rename Participant ID columns without spaces
age_at_scan_2.rename(columns={"BRICK-nummer" : "Participant_ID"}, inplace=True)
brick_gender.rename(columns={"Participant ID" : "Participant_ID"}, inplace=True)

In [ ]:
BRICK_volumetric_dems = pd.merge(age_at_scan_2, brick_gender, on="Participant_ID", how="left")
print(BRICK_volumetric_dems.head())

In [ ]:
#now merge this one with the volumetric data csv, drop first index column
brick_volumetric = pd.read_csv("/home/a.kidane@mydre.org/brickstudy/secret_data/brain_volumes_from_freesurfer_no_qc.csv", index_col=0)

In [ ]:
#rename first column and change hyphens into underscores
brick_volumetric.rename(columns={"Participant ID" : "Participant_ID"}, inplace=True)
brick_volumetric["Participant_ID"] = brick_gender["Participant_ID"].str.replace('-',"_")

In [ ]:
#merge the demographic data and demographic data

brick_volumetric_dems_00 = pd.merge(brick_volumetric,BRICK_volumetric_dems, on="Participant_ID", how="left")

print(brick_volumetric_dems_00.head())

In [ ]:
file_path = "/home/a.kidane@mydre.org/brickstudy/secret_data/brick_volumetric_dems_01.csv"
brick_volumetric_dems_00.to_csv(file_path, index=False)